In [269]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [270]:
#importing and visualising the data

data_initial=pd.read_csv('/content/Railway Ticket Confirmation.csv')

data_initial.head()

,PNR Number,Train Number,Date of Journey,Class of Travel,Quota,Source Station,Destination Station,Booking Date,Current Status,Number of Passengers,...,Booking Channel,Travel Distance,Number of Stations,Travel Time,Train Type,Seat Availability,Special Considerations,Holiday or Peak Season,Waitlist Position,Confirmation Status
0,PNR0000000000,51450,2024-09-01,3AC,General,NDLS,CSMT,2024-01-01,Confirmed,4,...,Counter,1656,17,37,Shatabdi,159,Senior Citizen,Yes,NaN,Confirmed
1,PNR0000000001,54807,2024-09-02,3AC,Premium Tatkal,MMCT,LTT,2024-01-02,Waitlisted,5,...,Mobile App,1932,18,6,Shatabdi,309,NaN,Yes,WL097,Not Confirmed
2,PNR0000000002,14396,2024-09-03,3AC,Ladies,GKP,BBS,2024-01-03,RAC,5,...,IRCTC Website,155,4,17,Express,143,NaN,Yes,NaN,Confirmed
3,PNR0000000003,20295,2024-09-04,3AC,Ladies,ASR,KOAA,2024-01-04,Waitlisted,1,...,Counter,1840,5,16,Superfast,256,Senior Citizen,No,WL011,Not Confirmed
4,PNR0000000004,48598,2024-09-05,2AC,Tatkal,MAS,SBC,2024-01-05,Confirmed,3,...,Mobile App,1766,9,32,Express,58,NaN,Yes,NaN,Confirmed


In [271]:
data_initial.columns

Index(['PNR Number', 'Train Number', 'Date of Journey', 'Class of Travel',
       'Quota', 'Source Station', 'Destination Station', 'Booking Date',
       'Current Status', 'Number of Passengers', 'Age of Passengers',
       'Booking Channel', 'Travel Distance', 'Number of Stations',
       'Travel Time', 'Train Type', 'Seat Availability',
       'Special Considerations', 'Holiday or Peak Season', 'Waitlist Position',
       'Confirmation Status'],
      dtype='object')

In [272]:
#Preprocessing
#1 Dropping unimportant columns
drop_columns=['PNR Number','Age of Passengers','Booking Channel','Travel Distance','Number of Stations','Travel Time']

data_initial.drop(columns=drop_columns,inplace=True)
data_initial.head()

,Train Number,Date of Journey,Class of Travel,Quota,Source Station,Destination Station,Booking Date,Current Status,Number of Passengers,Train Type,Seat Availability,Special Considerations,Holiday or Peak Season,Waitlist Position,Confirmation Status
0,51450,2024-09-01,3AC,General,NDLS,CSMT,2024-01-01,Confirmed,4,Shatabdi,159,Senior Citizen,Yes,NaN,Confirmed
1,54807,2024-09-02,3AC,Premium Tatkal,MMCT,LTT,2024-01-02,Waitlisted,5,Shatabdi,309,NaN,Yes,WL097,Not Confirmed
2,14396,2024-09-03,3AC,Ladies,GKP,BBS,2024-01-03,RAC,5,Express,143,NaN,Yes,NaN,Confirmed
3,20295,2024-09-04,3AC,Ladies,ASR,KOAA,2024-01-04,Waitlisted,1,Superfast,256,Senior Citizen,No,WL011,Not Confirmed
4,48598,2024-09-05,2AC,Tatkal,MAS,SBC,2024-01-05,Confirmed,3,Express,58,NaN,Yes,NaN,Confirmed


In [273]:
data_initial.shape

(30000, 15)

In [274]:
#2 checking if there are any null or missing values
data_initial.isnull().sum()

,0
Train Number,0
Date of Journey,0
Class of Travel,0
Quota,0
Source Station,0
Destination Station,0
Booking Date,0
Current Status,0
Number of Passengers,0
Train Type,0


In [275]:
# replacing null in special considerations and waitlist positon to 0
data_initial['Special Considerations'].fillna(0, inplace=True)

data_initial['Waitlist Position'].fillna(0, inplace=True)

data_initial.isnull().sum()

<ipython-input-275-bd43b6a7553b>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_initial['Special Considerations'].fillna(0, inplace=True)
<ipython-input-275-bd43b6a7553b>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

,0
Train Number,0
Date of Journey,0
Class of Travel,0
Quota,0
Source Station,0
Destination Station,0
Booking Date,0
Current Status,0
Number of Passengers,0
Train Type,0


In [276]:
#3 Converting the strings to numbers so that our algorithm can work on it

string_columns=data_initial.select_dtypes(include=['object']).columns
string_columns



Index(['Date of Journey', 'Class of Travel', 'Quota', 'Source Station',
       'Destination Station', 'Booking Date', 'Current Status', 'Train Type',
       'Special Considerations', 'Holiday or Peak Season', 'Waitlist Position',
       'Confirmation Status'],
      dtype='object')

In [277]:
data_initial.columns

Index(['Train Number', 'Date of Journey', 'Class of Travel', 'Quota',
       'Source Station', 'Destination Station', 'Booking Date',
       'Current Status', 'Number of Passengers', 'Train Type',
       'Seat Availability', 'Special Considerations', 'Holiday or Peak Season',
       'Waitlist Position', 'Confirmation Status'],
      dtype='object')

In [278]:

data_initial['Waitlist Position'] = data_initial['Waitlist Position'].replace('WL', '', regex=True)
data_initial['Waitlist Position'] = data_initial['Waitlist Position'].astype(int)
data_initial

,Train Number,Date of Journey,Class of Travel,Quota,Source Station,Destination Station,Booking Date,Current Status,Number of Passengers,Train Type,Seat Availability,Special Considerations,Holiday or Peak Season,Waitlist Position,Confirmation Status
0,51450,2024-09-01,3AC,General,NDLS,CSMT,2024-01-01,Confirmed,4,Shatabdi,159,Senior Citizen,Yes,0,Confirmed
1,54807,2024-09-02,3AC,Premium Tatkal,MMCT,LTT,2024-01-02,Waitlisted,5,Shatabdi,309,0,Yes,97,Not Confirmed
2,14396,2024-09-03,3AC,Ladies,GKP,BBS,2024-01-03,RAC,5,Express,143,0,Yes,0,Confirmed
3,20295,2024-09-04,3AC,Ladies,ASR,KOAA,2024-01-04,Waitlisted,1,Superfast,256,Senior Citizen,No,11,Not Confirmed
4,48598,2024-09-05,2AC,Tatkal,MAS,SBC,2024-01-05,Confirmed,3,Express,58,0,Yes,0,Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,38801,2106-10-17,3AC,Tatkal,BCT,CSMT,2106-02-15,Confirmed,5,Rajdhani,409,0,No,0,Confirmed
29996,99609,2106-10-18,1AC,General,MMCT,NJP,2106-02-16,RAC,4,Shatabdi,90,Senior Citizen,No,0,Confirmed
29997,46802,2106-10-19,Sleeper,General,JP,MAS,2106-02-17,Waitlisted,3,Shatabdi,110,0,Yes,97,Not Confirmed
29998,94878,2106-10-20,Sleeper,General,BSB,CNB,2106-02-18,Waitlisted,4,Express,91,0,No,6,Not Confirmed


In [279]:
import random
from datetime import datetime, timedelta
import calendar

# Function to generate a random Booking Date within the 1990-2023 range
def randomize_booking_date():
    year = random.randint(1990, 2023)
    month = random.randint(1, 12)  # Random month between 1 and 12
    day = random.randint(1, 28)  # Random day between 1 and 28 to avoid month-day mismatch
    try:
        return datetime(year, month, day).strftime('%Y-%m-%d')
    except ValueError:
        return randomize_booking_date()  # Retry if the date is invalid (e.g., February 30)

# Function to generate Date of Journey based on the Booking Date
def generate_journey_date(booking_date_str):
    booking_date = datetime.strptime(booking_date_str, '%Y-%m-%d')

    # Ensure the Date of Journey is between 2 to 4 months after the Booking Date
    delta_days = random.randint(3, 120)  # Between 2 and 4 months (approx. 30 days per month)
    journey_date = booking_date + timedelta(days=delta_days)

    # Ensure the Date of Journey is within the 1990-2023 range
    if journey_date.year < 1990 or journey_date.year > 2023:
        # Randomize year while ensuring the new date is valid
        new_year = random.randint(1990, 2023)
        # Make sure the new month and day are valid in the new year
        new_month = journey_date.month
        new_day = min(journey_date.day, calendar.monthrange(new_year, new_month)[1])  # Avoid invalid day
        journey_date = journey_date.replace(year=new_year, month=new_month, day=new_day)

    return journey_date.strftime('%Y-%m-%d')

# Apply the function to generate random Booking Date and Date of Journey
data_initial['Booking Date'] = [randomize_booking_date() for _ in range(len(data_initial))]
data_initial['Date of Journey'] = data_initial['Booking Date'].apply(generate_journey_date)

# Preview the dataset with updated dates
data_initial[['Booking Date', 'Date of Journey']].head(20)

,Booking Date,Date of Journey
0,2009-02-26,2009-04-01
1,2008-05-07,2008-06-01
2,1993-01-24,1993-03-01
3,1999-05-08,1999-06-29
4,2016-02-18,2016-03-18
5,2022-04-28,2022-08-15
6,2018-01-27,2018-03-28
7,1998-06-23,1998-09-17
8,1997-12-23,1997-12-29
9,2003-12-01,2004-03-07


In [280]:
#data_initial[data_initial['Current Status'] == 'Waitlisted']['Confirmation Status'].value_counts()
data_initial[data_initial["Waitlist Position"]!=0]['Confirmation Status'].value_counts()

,count
Confirmation Status,
Not Confirmed,10053


In [281]:
data_initial[data_initial["Current Status"]=="Waitlisted"].value_counts()

Train Number  Date of Journey  Class of Travel  Quota           Source Station  Destination Station  Booking Date  Current Status  Number of Passengers  Train Type  Seat Availability  Special Considerations  Holiday or Peak Season  Waitlist Position  Confirmation Status
10002         1999-07-18       Sleeper          Ladies          ASR             UMB                  1999-05-21    Waitlisted      4                     Shatabdi    112                Senior Citizen          Yes                     194                Not Confirmed          1
70165         1998-01-22       3AC              Ladies          UMB             SC                   1997-11-18    Waitlisted      4                     Express     488                Defense Quota           Yes                     35                 Not Confirmed          1
70107         2005-09-06       Sleeper          Tatkal          HWH             JHS                  2005-06-01    Waitlisted      4                     Rajdhani    17                 Senior Citizen          No                      83                 Not Confirmed          1
70112         2014-12-01       2AC              General         BBS             HWH                  2014-10-11    Waitlisted      3                     Express     322                Senior Citizen          Yes                     33                 Not Confirmed          1
70137         2019-12-29       3AC              Tatkal          BSB             CSMT                 2019-11-28    Waitlisted      1                     Shatabdi    130                Defense Quota           Yes                     145                Not Confirmed          1
                                                                                                                                                                                                                                                                                 ..
40244         2004-11-18       3AC              Tatkal          JU              JP                   2004-08-07    Waitlisted      5                     Shatabdi    256                0                       Yes                     68                 Not Confirmed          1
              2004-12-30       2AC              General         JU              BCT                  2004-12-06    Waitlisted      3                     Rajdhani    276                Defense Quota           No                      200                Not Confirmed          1
40304         1993-08-25       2AC              General         UMB             KOAA                 1993-07-06    Waitlisted      1                     Superfast   406                Senior Citizen          No                      38                 Not Confirmed          1
40313         2013-06-20       2AC              General         LKO             MAS                  2013-05-13    Waitlisted      4                     Express     464                Defense Quota           No                      69                 Not Confirmed          1
99997         2017-10-16       Sleeper          Premium Tatkal  JHS             BBS                  2017-07-12    Waitlisted      5                     Rajdhani    448                Defense Quota           Yes                     132                Not Confirmed          1
Name: count, Length: 10053, dtype: int64

In [282]:
quota_unique = data_initial['Quota'].unique()
class_of_travel_unique = data_initial['Class of Travel'].unique()
special_considerations_unique = data_initial['Special Considerations'].unique()

# Print the unique values for each column
print("Unique values in 'Quota':")
print(quota_unique)
print("\nUnique values in 'Class of Travel':")
print(class_of_travel_unique)
print("\nUnique values in 'Special Considerations':")
print(special_considerations_unique)

Unique values in 'Quota':
['General' 'Premium Tatkal' 'Ladies' 'Tatkal']

Unique values in 'Class of Travel':
['3AC' '2AC' 'Sleeper' '1AC']

Unique values in 'Special Considerations':
['Senior Citizen' 0 'Defense Quota']


In [283]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

# Function to calculate confirmation probability based on custom conditions
def calculate_confirmation_probability(row):
    # Initialize base probability
    probability = 0.3  # 30% of waitlisted tickets are confirmed

    # Priority weights for Quota
    quota_weights = {
        'General': 0.4,
        'Premium Tatkal': 0.3,
        'Ladies': 0.2,
        'Tatkal': 0.1
    }

    # Priority weights for Class of Travel
    class_weights = {
        '3AC': 0.4,
        'Sleeper': 0.3,
        '2AC': 0.2,
        '1AC': 0.1
    }

    # Priority weights for Special Considerations
    special_consideration_weights = {
        'Defense Quota': 0.4,
        'Senior Citizen': 0.4,
        'No Special Considerations': 0.2
    }

    # Get the weight for the Quota
    quota = row['Quota']
    probability += quota_weights.get(quota, 0)  # Default to 0 if value is not found

    # Get the weight for the Class of Travel
    class_of_travel = row['Class of Travel']
    probability += class_weights.get(class_of_travel, 0)  # Default to 0 if value is not found

    # Get the weight for Special Considerations
    special_consideration = row['Special Considerations']
    probability += special_consideration_weights.get(special_consideration, 0)  # Default to 0 if value is not found

    # Date difference adjustment: If the Date of Journey is closer to the Booking Date, increase probability
    booking_date = datetime.strptime(row['Booking Date'], '%Y-%m-%d')
    journey_date = datetime.strptime(row['Date of Journey'], '%Y-%m-%d')
    date_diff = (journey_date - booking_date).days  # Difference in days

    # If travel date is closer (e.g., within 30 days), increase the probability
    if date_diff < 30:
        probability += 0.2  # Add 20% if travel date is within 30 days

    # Ensure probability is within the range [0, 1]
    probability = min(probability, 1.0)

    return probability


waitlisted_tickets = data_initial[data_initial['Current Status'] == 'Waitlisted'].copy()  # Explicit copy

# Calculate the confirmation probability for each row (based on Quota, Class of Travel, Special Considerations)
waitlisted_tickets.loc[:, 'Custom Probability'] = waitlisted_tickets.apply(calculate_confirmation_probability, axis=1)

# Assign probabilities inversely proportional to the waitlist position (higher waitlist position, lower probability)
waitlist_positions = waitlisted_tickets['Waitlist Position']
waitlist_probabilities = 1 / (waitlist_positions + 1)

# Normalize the waitlist probabilities to sum to 1
waitlist_probabilities = waitlist_probabilities / waitlist_probabilities.sum()

# Combine the custom probability (Quota, Class of Travel, Special Considerations, etc.)
# and the waitlist position probability (inversely proportional to position)
final_probabilities = 0.5 * waitlist_probabilities + 0.5 * waitlisted_tickets['Custom Probability']

# Normalize the final probabilities to sum to 1
final_probabilities = final_probabilities / final_probabilities.sum()

# Calculate the number of tickets to confirm (30% of waitlisted tickets)
num_to_confirm = int(len(waitlisted_tickets) * 0.3)

# Randomly select rows to confirm based on the final probabilities
confirmed_indices = np.random.choice(
    waitlisted_tickets.index,
    size=num_to_confirm,
    replace=False,
    p=final_probabilities
)

# Update the Confirmation Status for selected rows
data_initial.loc[confirmed_indices, 'Confirmation Status'] = 'Confirmed'

# Ensure others remain 'Not Confirmed'
data_initial.loc[~data_initial.index.isin(confirmed_indices) & (data_initial['Current Status'] == 'Waitlisted'), 'Confirmation Status'] = 'Not Confirmed'

# Preview the updated dataset
print(data_initial['Confirmation Status'].value_counts())


Confirmation Status
Confirmed        22962
Not Confirmed     7038
Name: count, dtype: int64


In [284]:
data_initial[data_initial["Waitlist Position"]!=0]['Confirmation Status'].value_counts()

,count
Confirmation Status,
Not Confirmed,7038
Confirmed,3015


In [285]:
data_initial.head()

,Train Number,Date of Journey,Class of Travel,Quota,Source Station,Destination Station,Booking Date,Current Status,Number of Passengers,Train Type,Seat Availability,Special Considerations,Holiday or Peak Season,Waitlist Position,Confirmation Status
0,51450,2009-04-01,3AC,General,NDLS,CSMT,2009-02-26,Confirmed,4,Shatabdi,159,Senior Citizen,Yes,0,Confirmed
1,54807,2008-06-01,3AC,Premium Tatkal,MMCT,LTT,2008-05-07,Waitlisted,5,Shatabdi,309,0,Yes,97,Confirmed
2,14396,1993-03-01,3AC,Ladies,GKP,BBS,1993-01-24,RAC,5,Express,143,0,Yes,0,Confirmed
3,20295,1999-06-29,3AC,Ladies,ASR,KOAA,1999-05-08,Waitlisted,1,Superfast,256,Senior Citizen,No,11,Not Confirmed
4,48598,2016-03-18,2AC,Tatkal,MAS,SBC,2016-02-18,Confirmed,3,Express,58,0,Yes,0,Confirmed


In [286]:
# Ensure 'Waitlist Position' is numeric
data_initial['Waitlist Position'] = pd.to_numeric(data_initial['Waitlist Position'], errors='coerce').fillna(0).astype(int)

# Convert all categorical columns to strings
categorical_columns = ['Class of Travel', 'Quota', 'Source Station', 'Destination Station',
                       'Current Status', 'Train Type', 'Special Considerations',
                       'Holiday or Peak Season', 'Confirmation Status']
for col in categorical_columns:
    data_initial[col] = data_initial[col].astype(str)

# Verify no mixed types remain
print(data_initial.dtypes)


Train Number               int64
Date of Journey           object
Class of Travel           object
Quota                     object
Source Station            object
Destination Station       object
Booking Date              object
Current Status            object
Number of Passengers       int64
Train Type                object
Seat Availability          int64
Special Considerations    object
Holiday or Peak Season    object
Waitlist Position          int64
Confirmation Status       object
dtype: object


In [333]:
import joblib
from sklearn.preprocessing import LabelEncoder

# Initialize a dictionary to store label encoders
label_encoders = {}

# Identify columns to encode (categorical data)
categorical_columns = data_initial.select_dtypes(include=['object']).columns

# Apply Label Encoding to all categorical columns
for column in categorical_columns:
    le = LabelEncoder()
    data_initial[column] = le.fit_transform(data_initial[column])
    label_encoders[column] = le  # Save the encoder for future use

    # Save the encoder using joblib
    joblib.dump(le, f'{column}_encoder.pkl')

# Preview the transformed DataFrame
data_initial.head()


,Train Number,Date of Journey,Class of Travel,Quota,Source Station,Destination Station,Booking Date,Current Status,Number of Passengers,Train Type,Seat Availability,Special Considerations,Holiday or Peak Season,Waitlist Position,Confirmation Status
16540,75542,3696,1,3,18,3,3485,2,3,3,383,2,0,77,0
19296,67297,221,0,3,11,9,191,2,5,2,87,2,0,60,1
2069,69185,10038,2,3,16,4,9416,1,1,2,39,0,1,0,0
13640,32066,8132,1,2,11,6,7570,0,4,1,38,0,1,0,0
17658,28211,5492,3,0,5,10,5120,0,2,1,263,2,0,0,0


In [288]:
data_initial.head(20)

,Train Number,Date of Journey,Class of Travel,Quota,Source Station,Destination Station,Booking Date,Current Status,Number of Passengers,Train Type,Seat Availability,Special Considerations,Holiday or Peak Season,Waitlist Position,Confirmation Status
0,51450,6355,2,0,17,6,5973,0,4,2,159,2,1,0,0
1,54807,6074,2,2,16,14,5721,2,5,2,309,0,1,97,0
2,14396,1031,2,1,7,2,960,1,5,0,143,0,1,0,0
3,20295,3119,2,1,1,12,2904,2,1,3,256,2,0,11,1
4,48598,8694,1,3,15,20,8147,0,3,0,58,0,1,0,0
5,79825,10837,1,2,12,16,10102,1,2,0,223,1,1,0,0
6,95438,9371,3,0,1,19,8760,1,4,3,256,2,1,0,0
7,55096,2860,0,1,21,9,2633,0,5,1,374,1,0,0,0
8,66367,2622,0,3,14,7,2482,1,1,1,62,0,0,0,0
9,49309,4684,0,2,13,15,4329,0,1,3,313,0,1,0,0


In [338]:
from sklearn.utils import shuffle

# Shuffle the dataset
data_initial = shuffle(data_initial, random_state=90)

# Split the data again after shuffling
X = data_initial.drop(columns=['Confirmation Status'])
y = data_initial['Confirmation Status'].apply(lambda x: 1 if x == 'Confirmed' else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
y.head(20)

,Confirmation Status
16355,0
7689,0
27841,0
29066,0
2676,0
14026,0
9286,0
22715,0
10761,0
2353,0


In [328]:
# Introduce randomness by flipping some target labels
flip_prob = 0.05  # 5% of the labels are flipped randomly
flip_mask = np.random.rand(len(y_train)) < flip_prob
y_train[flip_mask] = 1 - y_train[flip_mask]

In [329]:
# Add Gaussian noise to the features
noise_factor = 0.01  # You can adjust this value based on how much randomness you want
X_train_noisy = X_train + noise_factor * np.random.randn(*X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.randn(*X_test.shape)


In [336]:

# Count how many values in y are equal to 1
confirmed_count = (y == 1).sum()

# Print the result
print(f'Number of confirmed tickets (y == 1): {confirmed_count}')


Number of confirmed tickets (y == 1): 0


In [293]:

# Define a neural network model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9044 - loss: 70.6427 - val_accuracy: 0.9471 - val_loss: 37.8145
Epoch 2/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9060 - loss: 31.9743 - val_accuracy: 0.9469 - val_loss: 19.3148
Epoch 3/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8963 - loss: 21.8260 - val_accuracy: 0.9471 - val_loss: 13.7710
Epoch 4/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8917 - loss: 16.9157 - val_accuracy: 0.9471 - val_loss: 35.6640
Epoch 5/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9044 - loss: 21.6077 - val_accuracy: 0.4558 - val_loss: 20.7343
Epoch 6/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8994 - loss: 14.8836 - val_accuracy: 0.9469 - val_loss: 48.0263
Epoch 7/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9045 - loss: 15.4468 - val_accuracy: 0.9471 - val_loss: 22.4836
Epoch 8/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9002 - loss: 15.7030 - val_accuracy

In [294]:
# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}%")


188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 4.2083e-12
Test Accuracy: 1.0%


In [295]:
model.save('ticket_confirmation_model.h5')

In [296]:
import joblib
